# Этап анализа данных [Supermarket](https://www.kaggle.com/aungpyaeap/supermarket-sales?select=supermarket_sales+-+Sheet1.csv)

### Подключим необходимые библиотеки

In [52]:
import psycopg2
import psycopg2.extras as extras
import pandas as pd
import os
from pathlib import Path
import numpy as np
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta

### Подгрузим датасет в pandas DataFrame

In [21]:
file = Path("C:\\_projects\\education\\graduate\\data\\supermarket_sales.csv")

In [22]:
df = pd.read_csv(file, sep=',', encoding='utf8')

In [23]:
df.head()

,Invoice ID,Branch,City,Customer type,Gender,Product line,Unit price,Quantity,Tax 5%,Total,Date,Time,Payment,cogs,gross margin percentage,gross income,Rating
0,750-67-8428,A,Yangon,Member,Female,Health and beauty,74.69,7,26.1415,548.9715,1/5/2019,13:08,Ewallet,522.83,4.761905,26.1415,9.1
1,226-31-3081,C,Naypyitaw,Normal,Female,Electronic accessories,15.28,5,3.8200,80.2200,3/8/2019,10:29,Cash,76.40,4.761905,3.8200,9.6
2,631-41-3108,A,Yangon,Normal,Male,Home and lifestyle,46.33,7,16.2155,340.5255,3/3/2019,13:23,Credit card,324.31,4.761905,16.2155,7.4
3,123-19-1176,A,Yangon,Member,Male,Health and beauty,58.22,8,23.2880,489.0480,1/27/2019,20:33,Ewallet,465.76,4.761905,23.2880,8.4
4,373-73-7910,A,Yangon,Normal,Male,Sports and travel,86.31,7,30.2085,634.3785,2/8/2019,10:37,Ewallet,604.17,4.761905,30.2085,5.3


## Рассмотрим информацию о типах данных и количестве записей

In [24]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 17 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Invoice ID               1000 non-null   object 
 1   Branch                   1000 non-null   object 
 2   City                     1000 non-null   object 
 3   Customer type            1000 non-null   object 
 4   Gender                   1000 non-null   object 
 5   Product line             1000 non-null   object 
 6   Unit price               1000 non-null   float64
 7   Quantity                 1000 non-null   int64  
 8   Tax 5%                   1000 non-null   float64
 9   Total                    1000 non-null   float64
 10  Date                     1000 non-null   object 
 11  Time                     1000 non-null   object 
 12  Payment                  1000 non-null   object 
 13  cogs                     1000 non-null   float64
 14  gross margin percentage  

### Рассмотрим информацию о крайних и средних значениях, стандартном отклонении

In [25]:
df.describe()

,Unit price,Quantity,Tax 5%,Total,cogs,gross margin percentage,gross income,Rating
count,1000.000000,1000.000000,1000.000000,1000.000000,1000.00000,1.000000e+03,1000.000000,1000.00000
mean,55.672130,5.510000,15.379369,322.966749,307.58738,4.761905e+00,15.379369,6.97270
std,26.494628,2.923431,11.708825,245.885335,234.17651,6.131498e-14,11.708825,1.71858
min,10.080000,1.000000,0.508500,10.678500,10.17000,4.761905e+00,0.508500,4.00000
25%,32.875000,3.000000,5.924875,124.422375,118.49750,4.761905e+00,5.924875,5.50000
50%,55.230000,5.000000,12.088000,253.848000,241.76000,4.761905e+00,12.088000,7.00000
75%,77.935000,8.000000,22.445250,471.350250,448.90500,4.761905e+00,22.445250,8.50000
max,99.960000,10.000000,49.650000,1042.650000,993.00000,4.761905e+00,49.650000,10.00000


### Переименуем колонки для вставки в БД

In [26]:
df.columns = list(map(lambda x: x.replace(' ', '_').replace('%', '_percent').lower(), df.columns))

In [27]:
df.head()

,invoice_id,branch,city,customer_type,gender,product_line,unit_price,quantity,tax_5_percent,total,date,time,payment,cogs,gross_margin_percentage,gross_income,rating
0,750-67-8428,A,Yangon,Member,Female,Health and beauty,74.69,7,26.1415,548.9715,1/5/2019,13:08,Ewallet,522.83,4.761905,26.1415,9.1
1,226-31-3081,C,Naypyitaw,Normal,Female,Electronic accessories,15.28,5,3.8200,80.2200,3/8/2019,10:29,Cash,76.40,4.761905,3.8200,9.6
2,631-41-3108,A,Yangon,Normal,Male,Home and lifestyle,46.33,7,16.2155,340.5255,3/3/2019,13:23,Credit card,324.31,4.761905,16.2155,7.4
3,123-19-1176,A,Yangon,Member,Male,Health and beauty,58.22,8,23.2880,489.0480,1/27/2019,20:33,Ewallet,465.76,4.761905,23.2880,8.4
4,373-73-7910,A,Yangon,Normal,Male,Sports and travel,86.31,7,30.2085,634.3785,2/8/2019,10:37,Ewallet,604.17,4.761905,30.2085,5.3


### Подготовим класс PgAdapter, который позволит выполнять базовые функции с БД

In [33]:
class PgAdapter():
    def __init__(self):
        self.connection = None
        print('Adapter created')

    def connect(self, host, port, database, user, password):
        self.connection = psycopg2.connect(host=host, port=port, database=database, user=user, password=password)
        
    def select(self, command):
        cursor = self.connection.cursor()
        cursor.execute(command)
        df = pd.DataFrame(cursor)
        cursor.close()
        return df
    
    def execute(self, command):
        cursor = self.connection.cursor()
        cursor.execute(command)
        cursor.close()
        self.connection.commit()
    
    def insert(self, schema, table, data):
        tuples = [tuple(x) for x in data.to_numpy()]
        cols = ','.join (list(df.columns))
        query  = "INSERT INTO %s(%s) VALUES %%s" % (f"{schema}.{table}", cols)
        cursor = self.connection.cursor()
        extras.execute_values(cursor, query, tuples)
        cursor.close()
        self.connection.commit()

### Для подключения необходимо получить переменные окружения с credentials

In [ ]:
host = os.getenv("DB_HOST")
port = os.getenv("DB_PORT")
database = os.getenv("DB_DATABASE")
user = os.getenv("DB_USER")
password = os.getenv("DB_PASSWORD")
schema = "raw"
table = "supermarket_sales"

#### Выполним подключение

In [34]:
pg = PgAdapter()

Adapter created


In [35]:
pg.connect(host, port, database, user, password)

#### Создадим необходимые объекты

In [37]:
pg.execute(f"CREATE SCHEMA {schema} AUTHORIZATION {user};")

In [39]:
create_table = f"""
CREATE TABLE {schema}.{table} (
    id int8 NOT NULL GENERATED BY DEFAULT AS IDENTITY,
    invoice_id text NULL,
    branch text NULL,
    city text NULL,
    customer_type text NULL,
    gender text NULL,
    product_line text NULL,
    unit_price numeric NULL,
    quantity numeric NULL,
    tax_5_percent numeric NULL,
    total numeric NULL,
    "date" text NULL,
    "time" text NULL,
    payment text NULL,
    cogs numeric NULL,
    gross_margin_percentage numeric NULL,
    gross_income numeric NULL,
    rating numeric NULL
);
"""
pg.execute(create_table)

#### Подгрузим сырые данные в БД

In [64]:
pg.insert(schema, table, df)

In [ ]:
pg.select(f"select count(*) as rows_count from {schema}.{table}")

In [65]:
pg.connection.close()

## Итого, на этапе анализа данных

* Был выполнен анализ данных
* Переименованы заголовки полей, чтобы данные возможно было вставить в БД
* Создан класс адаптера для БД для осуществления базовых функций
* Подгружены "сырые" данные в БД
* В промышленных условиях "сырые" данные из разных систем попадают в хранилище с помощью специальных ETL процессов. Для таких процесоов могут быть использованы разные способы доставки сырых данных, например с помощью шины данных kafka.